# Analysis of numeric coverage in CSKG

This notebook:
* computes numbers of statements in CSKG that contain numeric information, either exact or approximate; either in English or as digits
* extracts various collections and stores them as KGTK files

## Analysis

In [23]:
import gzip
from tqdm import tqdm

In [7]:
graph='../output/cskg.tsv.gz'

In [1]:
units = [
    "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
    "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
    "sixteen", "seventeen", "eighteen", "nineteen",
]

tens = ["twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

scales = ["hundred", "thousand", "million", "billion", "trillion"]

In [2]:
keywords=set(units+tens+scales)

In [3]:
len(keywords)

33

In [20]:
def num_there(s, kw):
    return any(i.isdigit() for i in s) or any(k in s for k in kw)

In [15]:
def line2edge(l):
    return l.decode().strip().split('\t')

In [29]:
only_n1=[]
only_n2=[]
both=[]
with gzip.open(graph, 'r') as f:
    header=line2edge(next(f))
    for line in tqdm(f, total=6001531):
        edge=line2edge(line)
        n1_label=edge[4]
        n2_label=edge[5]
        
        num_n1=num_there(n1_label, keywords)
        num_n2=num_there(n2_label, keywords)

        if num_n1 and num_n2:
            both.append(line)
        elif num_n1:
            only_n1.append(line)
        elif num_n2:
            only_n2.append(line)


100%|██████████| 6001531/6001531 [00:58<00:00, 102782.90it/s]


In [37]:
print('BOTH HEAD AND TAIL CONTAIN NUMBERS:', len(both))
print('ONLY HEAD CONTAINS NUMBER:', len(only_n1)) 
print('ONLY TAIL CONTAINS NUMBER:',  len(only_n2))

BOTH HEAD AND TAIL CONTAIN NUMBERS: 78928
ONLY HEAD CONTAINS NUMBER: 174269
ONLY TAIL CONTAINS NUMBER: 144686


In [41]:
def write_to_file(filename, data, header):
    with open(filename, 'w') as w:
        w.write('\t'.join(header) + '\n')
        for line in data:
            w.write(line.decode())

In [42]:
write_to_file('../tmp/numeric_n1.tsv', only_n1, header)
write_to_file('../tmp/numeric_n2.tsv', only_n2, header)
write_to_file('../tmp/numeric_both.tsv', both, header)